# 96 - Su Doku

## Problem Statement

Su Doku (Japanese meaning <i>number place</i>) is the name given to a popular puzzle concept. Its origin is unclear, but credit must be attributed to Leonhard Euler who invented a similar, and much more difficult, puzzle idea called Latin Squares. The objective of Su Doku puzzles, however, is to replace the blanks (or zeros) in a 9 by 9 grid in such that each row, column, and 3 by 3 box contains each of the digits 1 to 9. Below is an example of a typical starting puzzle grid and its solution grid.

<div class="center">
<img src="images/p096_1.png" alt="p096_1.png" />     <img src="images/p096_2.png" alt="p096_2.png" /></div>

A well constructed Su Doku puzzle has a unique solution and can be solved by logic, although it may be necessary to employ "guess and test" methods in order to eliminate options (there is much contested opinion over this). The complexity of the search determines the difficulty of the puzzle; the example above is considered <i>easy</i> because it can be solved by straight forward direct deduction.

<p>The 6K text file, <a href="inputs/0096_sudoku.txt">sudoku.txt</a> (right click and 'Save Link/Target As...'), contains fifty different Su Doku puzzles ranging in difficulty, but all with unique solutions (the first puzzle in the file is the example above).</p>

By solving all fifty puzzles find the sum of the 3-digit numbers found in the top left corner of each solution grid; for example, 483 is the 3-digit number found in the top left corner of the solution grid above.

## Solution

We solve it with backtracking. Every time we encounter a zero, we try replacing it with another value. If this does not break the row, column and 3 by 3 box conditions, we go to the next zero. If a number is not valid, we check the next value. If none of the values is valid, we go back to the previously modified value and try to replace it with another value. We continue until the grid is entirely filled.

In [1]:
import re

def is_valid(grid, i, j):
    """ Check validity of sudoku grid after changing value at (i, j) """
    # Check rows and columns
    for k in range(len(grid)):
        if j != k and grid[i][k] == grid[i][j]:
            return False
        if i != k and grid[k][j] == grid[i][j]:
            return False
    # Check square
    i_ = (i // 3) * 3
    j_ = (j // 3) * 3
    for k in range(3):
        for l in range(3):
            if (i_ + k != i or j_ + l != j) and grid[i_ + k][j_ + l] == grid[i][j]:
                return False
    return True

def solve(grid, i, j):
    """ Solve the sudoku with backtracking and return top left 3 digit number """
    if i == len(grid):
        return 100 * grid[0][0] + 10 * grid[0][1] + grid[0][2]
    if grid[i][j] == 0:
        for k in range(1, 10):
            grid[i][j] = k
            if is_valid(grid, i, j):
                if j == 8:
                    res = solve(grid, i + 1, 0)
                else:
                    res = solve(grid, i, j + 1)
                if res:
                    return res
            grid[i][j] = 0
    else:
        if j == 8:
            res = solve(grid, i + 1, 0)
        else:
            res = solve(grid, i, j + 1)
        if res:
            return res

def extract_matrices_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    matrices = {}
    current_grid = None
    current_matrix = []

    for line in lines:
        line = line.strip()
        # Check if the line is a grid label
        if re.match(r'Grid \d{2}', line):
            if current_grid is not None and current_matrix:
                matrices[current_grid] = current_matrix
                current_matrix = []
            current_grid = line
        else:
            if line:  # Ignore empty lines
                current_matrix.append([int(char) for char in line])

    if current_grid is not None and current_matrix:
        matrices[current_grid] = current_matrix

    return matrices

In [2]:
file_path = 'inputs/0096_sudoku.txt'
matrices = extract_matrices_from_file(file_path)

res = 0
for matrix in matrices.values():
    res += solve(matrix, 0, 0)

res

24702